#1. opencv

##1.1 basic

In [ ]:
import cv2, sys
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imgBGR=cv2.imread(fileName, cv2.IMREAD_COLOR) #인수 안넣어주면 이미지 그대로 읽어옴
imgRGB=cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB) #color space를 바꾸어줌 #opencv는 bgr, matplotlib은 rgb
imgGray=cv2.imread(fileName, cv2.IMREAD_GRAYSCALE) #cv2.IMREAD_UNCHANGED로 읽어오면 png에서 alpha채널 읽어옴

plt.axis('off') #xy pixel 좌표 끔
plt.imshow(imgGray, cmap='gray',interpolation='bicubic',asepct='auto') # 알아서 interpolation을 해줌 default antialiased, 'auto' 비율 바꿀 수 있음
plt.show() #interpolation 이미지 해상도를 키웠을때 공백을 채워줌 기본적으로는 선형

cv2.namedWindow('img',cv2.WINDOW_FREERATIO) # WINDOW_FULLSCREEN
cv2.imshow('imgBGR',imgBGR) #namedwindow 안해주어도 알아서 창 만들고 띄어주긴함

loop=True
while (loop):
    inKey=cv2.waitKey() #무한 대기, 값 넣어주면 ms단위로 시간지나면 꺼짐 #기본적으로 키 입력 한 번 받는거라 while로 돌려주어야함
    if inKey==ord('q'):#ord(' ') #spacebar의 ascii, esc는 문자로 표현 안되어서 27로 직접넣음
        #cv2.destroyAllWindows()
        cv2.destroyWindow('img')
        loop=False

cv2.destroyAllWindows() # 열린 창 닫아주어야함, 부하걸림

cv2.imwrite('filename.jpg',imgBGR,[cv2.IMWRITE_JPEG_QUALITY,85]) #이미지 저장, jpg 정보 유지율

In [ ]:
cap=cv2.VideoCapture(0) # 0은 연결된 카메라 번호, 경로로 영상 지정

a1=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #왜 int?
a2=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

while(True):
    retval, frame = cap.read() # 호출할 때 마다 한 프레임씩 읽어옴
    if not retval:
        break
    cv2.imshow('frame',frame)

    key = cv2.waitKey(100) # 10fps니까 100ms 대기, 프레임 한장씩 넘기면서 영상 만들어줌
    if key==27: # ascii에서 esc가 27임
        break

if cap.isOpened():
    cap.release()

In [ ]:
cap=cv2.VideoCapture(0)

framesize=(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #tuple
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID') #codec
out1 = cv2.VideoWriter('data/record0.mp4',fourcc,fps,framesize)
out2 = cv2.VideoWriter('data/record2.mp4',fourcc,fps,framesize, isColor=False)

while (True):
    retval, frame = cap.read()

    if not retval:
        break

    out1.write(frame)

    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    out2.write(gray)

    cv2.imshow('frame',frame)
    cv2.imshow('gray',gray)

if cap.isOpened():
    cap.release()
    out1.release()
    out2.release()
    cv2.destroyAllWindows()

In [ ]:
hist = cv2.calcHist([src1],[0],None,[256],[0,256]) #np.ndarray
#이미지, 0번채널(grayscale), 마스크여부 ,hitsize 몇개로 나누는지 ,range
plt.plot(hist)
plt.show()

In [ ]:
#mask

cv2.copyTo(src,mask,dst)
#dst 출력영상 src와 크기 및 타입이 같은 dst를 넣어주면 새로 생성하지 않고 연산 수행
#그렇지 않으면 dst새로 생성해서 return

h,w=mask.shape[:2]
crop= dst[10:10+h, 10:10+w] # image size 보고 이미지 넣을 위치
# 얕은 복사라서 crop을 바꾸면 dst가 바뀜!!

dst3=cv2.copyTo(src,mask,crop) # mask연산은 mask크기만큼만
#src,mask,crop은 다 같은 크기여야함

In [ ]:
#hsv 색공간에서 영역을 검출해서 합성 rgb로는 어디까지가 녹색인지 명확히 하기힘듬
#색공간을 바꾸어서 채도,명도 조절 및 명확한 연산이 가능
hsv= cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV)

##1.2 filter

In [ ]:
dst=cv2.add(src,50) #포화연산, rgb채널에서는 밝기조절역할
dst=cv2.substract(src,50) #포화연산
dst=cv2.multiply(src,2) # 차이가 벌어져서 대비가 뚜렷해짐
dst=cv2.divide(src,src2,scale=255) # 나눈 결과에 255을 곱해서 밝기보정
dst=cv2.divide(gray, blr, scale=255)# blr된 것으로 나누어서 255를 곱해주면 엣지는 원래값보다 작은 값으로 나누어지고 255를 곱해주면 선명해짐

In [ ]:
dst= cv2.addWeighted(src1, alpha=0.9, src2, beta=0.1, gamma=0) #gamma 가중합 결과에 더해줄 상수항,밝기조절, 포화연산

#blending
sharpened = cv2.addWeighted(img, 1.5, bilateral, -0.5, 0) #blur된 이미지 즉 엣지부분이 사라진 저주파부분만 빼주면서 고주파(엣지)가 강조됨

In [ ]:
cv2.bilateralFilter(src,-1,10,5) # 주변의 픽셀을 사용해서 연산을 부드럽게 해줌, 엣지(선)는 살리고 안 면부분 피부같은 것이 부드러워짐
#양방향 필터는 픽셀 간의 공간적 근접성과 강도 유사성(비슷한 밝기,색상에 가중치 더 줌)을 모두 고려하여 작동한다.
#d: 필터직경, 필터가 고려하는 주변 픽셀들의 거리, -1을 넣으면 sigmaSpace 기반으로 자동으로 계산, 보통 5~9 사
#sigmaColor: 색 공간에서의 필터 강도. 이 값이 클수록 색상차이가 큰 색상도 고려하여 평균을 계산 즉 엣지가 더 흐려
#sigmaSpace: 공간 거리에서의 필터 강도. 값이 클수록 더 멀리 있는 픽셀들도 필터에 포함
#디테일이 많은 이미지에서는 d와 sigmaColor값을 줄인다.

In [ ]:
cv2.blur(src,(3,3)) #kernal size
cv2.GaussianBlur(src,(3,3),0) #kernal size, sigmaX, simgaX(분산)로 blurring 조절
# kernal size(0,0)하면 sigmax에 맞춰서 자동, 반대도 같은 논리, sigmaY생략 시 sigmaX=sigmaY
dst=cv2.medianBlur(src,3) #image는 주변 픽셀간의 큰 차이가 나는 경우가 거의 없다 서서히 변하는게 보통 논리

In [ ]:
cv2.fastNlMeansDenoisingColored(src, None, 10, 10, 7, 21)
#h=10 밝기채널 노이즈 제거 필터 강도
#hColor=10 컬러 이미지 채널간의 노이즈 제거 강도 보통 h와 같거나 작은값, 색상정보가 보통 더 민감하고 너무 크게 필터링하면 색상간의 경계가 흐려져서 색상번짐
#templateWindowSize=7: 노이즈 제거에 사용할 주변 픽셀들의 크기, searchWindowSize=21: 픽셀 주변에서 유사한 패턴을 찾을 검색 영역의 크기

In [ ]:
#high-pass filter
mask = np.asarray([[0,-0.5,0],[-0.5,3,-0.5],[0,-0.5,0]], dtype = np.float32) #모든 값의 합이 1이므로 전체 이미지의 밝기 유지
sharpening_img = cv2.filter2D(dst,-1, mask)

In [ ]:
src = cv2.imread('road.png',cv2.IMREAD_GRAYSCALE)
dst = cv2.Canny(src,64,128) # threshold 2개를 주어야함, 윤곽 읽어올 때 용이
# 검출하고싶은부분만 마스킹해서 딴 다음에 차선 주변의 노이즈만 처리

In [ ]:
bgr_planes=cv2.split(src) #bgr채널 분리
filtered_img = cv2.merge([bgr[0], bgr[1], bgr[2]]) # 합치기

In [ ]:
#선형변환
cv2.convertScaleAbs(src, alpha=1.1, beta=10)
# alpha Contrast control (1.0-3.0), beta Brightness control (-100 to 100)

In [ ]:
#contrast
cv2.normalize(src,None,0,255,cv2.NORM_MINMAX)
#없는 정보를 만드는것이 아니라 정보간격을 최대한 벌려서(분포를 바꾸어서) 사람눈으로 식별 용이하게 만듬
dst = cv2.equalizeHist(src)
#정규화는 일정한 간격으로 벌려주는 것, equalizerHist는 데이터 수치가 높은 구간은 간격이 넓게 수치가 적은부분은 간격이 좁게 중요한것이 더 대비되게, edge가 더 선명하게 살아남

In [ ]:
cv2.bitwise_and() #같은 위치의 두 픽셀의 비트가 모두 1인 경우 1, 두 값이 모두 1이 아닐 경우 0
cv2.bitwise_and(image, image, mask=mask) #mask 흰색 부분만 bitwise_and 연산 수행
cv2.bitwise_or() #같은 위치의 두 픽셀 중 한 개가 1인 경우 1, 두 값이 모두 1이 아닐 경우 0
cv2.bitwise_xor() #같은 위치의 두 픽셀의 값이 다른 경우 1, 같으면 0
cv2.bitwise_not() #비트, 이미지 색상 반
dst = cv2.resize(dst, (w, h), interpolation=cv2.INTER_NEAREST)

In [ ]:
# 대비조절, np.float32, np.uint8 바꾸어주기
img_float = result.astype(np.float32) / 255.0
img_float = img_float * 0.7
img_adjusted = (img_float * 255).astype(np.uint8)

In [ ]:
#unsharp masking
kernel_size=(5,5) # 블러의 범위. (야간 사진에서는 3,3의 작은 사이즈)-> 이미 조도가 낮아 노이즈가 많고 디테일이 부족한 상황 -> 디테일 손실을 최소화해야함.
sigma = 1.0 # Gaussian blur의 강도
amount = 1.0 # 선명도의 강도, 높을수록 선명해지지만 노이즈도 증가
blurred = cv2.GaussianBlur(image, kernel_size, sigma)
sharpened = cv2.addWeighted(image, 1 + amount, blurred, -amount, 0)

In [ ]:
threshold, mask = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)# 100:threshold, 255:threshold넘었을 때 넣어줄 값
#cv2.THRESH_BINARY: 픽셀 값이 임계값을 넘으면 maxval, 넘지 않으면 0 / cv2.THRESH_BINARY_INV 반대논리

In [ ]:
src_th=cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,7)
# 임계값을 주변 픽셀의 평균으로 적용하여 이미지 내 조명 변화에 맞추어 필터 적용
#maxvalue, blocksize:주변의 51개의 블록을 본다는것(51x51)-해상도에 따라 선택
#7 평균값에서 빼줄 c 상수값 c를 키울 수록 threshold가 작아져서 많은 픽셀이 255가 됨.

In [ ]:
# 마스크를 이용하여 노이즈 제거된 이미지 합성
dn1_img = np.where(mask[:, :, np.newaxis] == 255, dn1, src)

In [ ]:
# saturation
saturation_factor = 0.7 # 채도 감소 비율 (0 ~ 1 사이의 값, 0: 완전 흑백, 1: 원본 채도)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)# HSV 색 공간으로 변환
hsv[:, :, 1] = hsv[:, :, 1] * saturation_factor
saturation_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)# 다시 BGR 색 공간으로 변환

##1.3 input받아오기

In [ ]:
# Trackbar
def on_trackbar(pos): #인수로 pos를 받아오기 때문에 trackbar가 1개이면 getTrackbarPos없이 그 위치 값 그대로 쓸 수 있음
#두 개이상부터는 입력받은 pos가 어떤 trackbar인지 식별필요
    hmin = cv2.getTrackbarPos('H_min','Trackbar') # 마우스 입력을 받아옴
    hmax = cv2.getTrackbarPos('H_max','Trackbar')

    # inrange함수 적용
    dst = cv2.inRange(src, (hmin,0,0),(hmax,255,255)) # 두 threshold 사이에 있으면 1 아니면 0 바이너리로 리턴해줌
    cv2.imshow('Trackbar',dst)

cv2.namedWindow('Trackbar') # 창에 트랙바를 넣으려면 일단 창이 있어야함

#call-back 함수 어떤 이벤트가 발생했을 때 이벤트에 따라 호출되는 함수
cv2.createTrackbar('H_min','Trackbar',0,180,on_trackbar)
cv2.createTrackbar('H_max','Trackbar',180,180,on_trackbar)
cv2.setTrackbarPos('H_min','Trackbar',40) #마우스 입력대신 직접 값 넣어줄 때

##1.4 image making

In [ ]:
img = np.full((400,400,3),255, np.uint8) # 400x400x3 공간을 전부 255로 채움 흰색캔버스

cv2.putText(img, text, (50,350),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,255),2,cv2.LINE_AA)
#(50,350)입력 좌표 텍스트박스 좌측하단기준, 3 fontsize, (0,0,255) 색깔, 2 font 두께

pt1=(50,100)
pt2=(img.shape[0]-50, img.shape[1]-100)
cv2.line(img, pt1, pt2, (0,0,255),3,cv2.LINE_4)
# linetype8 은 대각선은 좀 표면이 거칠음 AA가 부드러움
# 꺼끌꺼끌해지면 연산량 적어짐 부드러우면 선을 많이 긋는거라 연산많아짐
# 보통 글씨 쓸때는 AA 많이씀
qt1=(30,70)
qt2=(50,100)
# (x1,y1) (x2,y2)
cv2.rectangle(img, qt1, qt2, (0,0,255), 1, cv2.LINE_4) #두께를 -1로 하면 안을 채움
# x,y,width,height
cv2.rectangle(img, (30,70,100,100),(255,0,0), 1, cv2.LINE_4)

cv2.circle(img, qt1, 30 ,(0,255,0),2,cv2.LINE_AA) # 좌표값은 int 여야함. 픽셀위치니까

##1.5 etc

In [ ]:
dst1 = np.clip(src+100,0,255).astype(np.uint8) # 범위를 0에서 255 지정하고 연산 수행, 최소,최대치 한정, 0~255 사이에 위치하도록 안전망 역할

In [ ]:
pixmin, pixmax,a,b = cv2.minMaxLoc(src) # a,b 최소 최대 위치

In [ ]:
colors=['b','g','r']
bgr_planes=cv2.split(src) # tuple형태로 나옴, numpy array배열이 하나씩 나누어져서 나옴

for (p,c) in zip(bgr_planes,colors):
  hist=cv2.calcHist([p],[0],None,[256],[0,256])
  plt.plot(hist,color=c)#color 그래프 그릴때 색상 지정

In [ ]:
cv2_imshow() #colab

In [ ]:
cv2.countNonZero() # 0이 아닌 픽셀의 개수, grayscale

In [ ]:
kernel = np.ones((5, 5), np.uint8) # (5,5)배열을 1로 채움

In [ ]:
while True:
    #waitkey + extension키 입력까지 받아들임 , 특수키 ascii코드가 없는
    key= cv2.waitKeyEx(30)
    if key == 0x1B: #esc(16진수 hexa)
        break
    elif key == 0x270000: #right key #6자리중에 2자리만 씀 waitkey보다 key의 길이가 길기 때문에 더 무거움
        direction=0
        x+=10
    elif key == 0x280000: #down key
        direction=1
        y+=10
    elif key == 0x250000: #left key
        direction=2
        x-=10
    elif key == 0x260000: #up key
        direction=3

In [ ]:
def mouse_callback(event, x, y, flags, param):
    #global img 함수 안에서도 img수정가능하게
    img=param[0] #parameter 인수로 받아서 전역변수설정안하고 img값 얻어옴

    if event==cv2.EVENT_LBUTTONDOWN:
      print('1')
    elif event==cv2.EVENT_LBUTTONUP:
      print('2')
    elif event==cv2.EVENT_MOUSEMOVE:
      print(x,y)#누른채로 움직여도 좌표 출력, 누른건 한번이고 움직이면 새로운 이벤트라서 콜백함수가 새로 호출되고 1은 나오지않고 좌표만 찍힘


    cv2.imshow('img',img) # 그림화면을 업데이트

# 흰색 캔버스 생성
#img = np.zeros((512,512,3),np.uint8)+255
img = np.ones((512,512,3),np.uint8)*255
#img = np.full((512, 512, 3), 255, dtype=np.uint8)

# main에서 setMouseCallback 함수를 실행하면서 콜백함수를 지정
cv2.namedWindow('img')
cv2.setMouseCallback('img',mouse_callback, [img])# [img]가 parameter, [img, param2, param3] 여러개주고 param[0]지정해서 불러오기 가능

cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# perspective 변환
h,w = img.shape[:2]
srcQuad=np.array([[220, 128], [469, 147], [378, 481], [47, 391]], np.float32)
dstQuad=np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]],np.float32) # image indexing 0부터 시작하니 w-1
pers =cv2.getPerspectiveTransform(srcQuad,dstQuad) #변환 행렬을 만들어줌
dst = cv2.warpPerspective(img,pers,(w,h)) #여긴 또 width height네

In [ ]:
# Affine 변환, warpAffine
def translate(src,x=0,y=0):
    if src is None:
        sys.exit('load fail')

    # 이미지의 이동변환
    aff = np.array([[1,0,x],[0,1,y]],dtype=np.float32)

    dst = cv2.warpAffine(src,aff,(0,0)) #(0,0)변환후에 출력되는 output크기 dsize,
    #(0,0)input이미지 크기 그대로 출력 아닌거같은데 scaling값은 알아서 계산해줌 docu 출처
    return dst

def shear(src,x=0,y=0,):
    h,w = src.shape[:2]
    if x>0 and y==0:
        aff = np.array([[1,x,0],[0,1,0]],dtype=np.float32)
        dst= cv2.warpAffine(src,aff,(w+int(h*x),h))
    if x==0 and y>0:
        aff = np.array([[1,0,0],[y,1,0]],dtype=np.float32)
        dst= cv2.warpAffine(src,aff,(w,h+int(w*y)))

    return dst

def scaling(src,x=1,y=1):
    h, w = src.shape[:2]
    aff= np.array([[x,0,0],[0,y,0]],dtype=np.float32)
    dst=cv2.warpAffine(src,aff,(w*x,h*y))
    return dst

def rotate(src,theta): #중심축은 기본적으로 좌측상단, (0,0)
    theta_rad = np.deg2rad(theta) #라디안 값으로 바꿔줘야함
    theta_rad = theta*math.pi/180
    aff=np.array([[np.cos(theta_rad),np.sin(theta_rad),0],[-np.sin(theta_rad),np.cos(theta_rad),0]])
    dst=cv2.warpAffine(src,aff,(0,0))
    return dst

def rotate2(src,theta):
    h,w=src.shape[:2]

    center=w/2,h/2

    rot =cv2.getRotationMatrix2D(center,angle=theta,scale=1)
    dst =cv2.warpAffine(src,rot,(0,0))
    return dst

In [ ]:
#resize
dst=cv2.resize(src,(0,0),fx=1.5,fy=1.5)# 비율 설정
dst1=cv2.resize(src,(1024,1024),interpolation=cv2.INTER_LANCZOS4)# 3개중에 가장 오래걸림 1등
dst2=cv2.resize(src,(1024,1024),interpolation=cv2.INTER_CUBIC)# 해상도 설정 inter_nearest보다 엣지가 깍두기 없이 부드럽고 2등
dst3=cv2.resize(src,(1024,1024),interpolation=cv2.INTER_NEAREST)# 3등